## 3. Feature Generation

In [2]:
#Switch to '/evanleungc/PHBS_TQFML/Project/Code' Repository becuase some classes and functions are in the directory
import os
os.chdir('/Users/apple/Documents/PHBS_TQFML/Project/Code')
import numpy as np
import pandas as pd
import os
import feature_funcs as ff
from HfqShare.Feature import Feature

In [3]:
#Switch to '/Data' repository to get data
current_path = os.getcwd()
data_path = os.path.dirname(os.getcwd()) + '/Data/'

In [ ]:
path = data_path + 'Stk_5F_2018/Stk_5F_2018/'
funcs = Feature(path)
funcs.apply_feature(feature_name = ['buyprice', 'high', 'low', 'canbuy', 'growth_rate', 'amplitude', 'above_mean'],\
	intra = (ff.get_buy, ff.get_high, ff.get_low, ff.get_canbuy, ff.growth_rate, ff.amplitude, ff.above_mean))
df1 = funcs.gen_train_data()
path = path + 'Stk_5F_2017/Stk_5F_2017/'
funcs = Feature(path)
funcs.apply_feature(feature_name = ['buyprice', 'high', 'low', 'canbuy', 'growth_rate', 'amplitude', 'above_mean'],\
	intra = (ff.get_buy, ff.get_high, ff.get_low, ff.get_canbuy, ff.growth_rate, ff.amplitude, ff.above_mean))
df2 = funcs.gen_train_data()
df = pd.concat([df1,df2])
lag = 2
df = ff.take_lag(df, 'buyprice', lagnum = lag)
df = ff.take_lag(df, 'buyprice', lagnum = 1)
df = ff.take_lag(df, 'canbuy', lagnum = lag)
df['target'] = [0 for i in range(len(df))]
df['buyret'] = df['high'] / df['lag_%s_buyprice'%lag] - 1
df['risk'] = df['low'] / df['lag_%s_buyprice'%lag] - 1
df['target'][(df['buyret'] > 0.03) & (df['risk'] > -0.02) & (df['lag_%s_canbuy'%lag] == 1)] = 1

<span id="jump_1">The following is the key variables description</span>

In [7]:
df.tail()

,above_mean,amplitude,buyprice,canbuy,code,date,growth_rate,high,lag_1_code,low,lag_2_buyprice,lag_1_buyprice,lag_2_canbuy,target,buyret,risk
742559,0.0,0.046162,37.69,1.0,300659,2017/11/07,0.019940,38.30,300659.0,36.61,36.25,37.59,1.0,1,0.056552,0.009931
742560,0.0,0.053044,39.50,1.0,300659,2017/11/08,0.027683,40.30,300659.0,38.27,37.59,37.69,1.0,1,0.072094,0.018090
742561,1.0,0.086293,41.11,1.0,300659,2017/11/09,0.056906,41.29,300659.0,38.01,37.69,39.50,1.0,1,0.095516,0.008490
742562,0.0,0.066001,41.97,1.0,300659,2017/11/10,0.021189,43.77,300659.0,41.06,39.50,41.11,1.0,1,0.108101,0.039494
742563,1.0,0.031866,40.37,1.0,300659,2017/11/13,0.010526,40.80,300659.0,39.54,41.11,41.97,1.0,0,-0.007541,-0.038190


* **buyprice:** It is the highest price in the last 20 miniutes of a trading day. We are going to assume we buy at this price.
* **canbuy:** If the stock has reached the price ceiling or floor, we assume that we cannot buy this stock and the variable will be 0. Otherwise, we assume that we can buy this stock and the variable will be 1. When doing the training, we will exclude all the samples with canbuy == 0.
* **buyret:** The return of buying at buyprice and sell in the highest price in the next two days.
* **risk:** The loss of buying at buyprice and sell in the lowest price in the next two days.
* **target:** The training target we use in this project. If in next twodays, the buyret > 0.03 and risk < -0.02, we consider it as a low-risk profitable trading opportunity and we set the label to be 1. Otherwise, we set it to be 0. 

Within the variables, there are two features we construct from the 5-min frequency data
* **amplitude:** high / low - 1, which indicates the fluctuation of the price
* **above_mean:** whether the last close price is above the 

### 3.2. Generate High-Frequency Features

In [8]:
hfqpath = data_path + 'group_data/'
hfqamount = pd.DataFrame()
for i in os.listdir(hfqpath):
    if i[0:5] == 'group':
        hfqamount = hfqamount.append(pd.read_csv(hfqpath + i, index_col = 0))
lag = 2
hfqamount['code'] = hfqamount['code'].map(ff.convert_stock)
hfqamount = hfqamount.drop('pchange_whole', axis = 1)
df['code'] = df['code'].map(ff.convert_stock)
df['date'] = df['date'].map(ff.convert_date)
mergedf = pd.merge(df, hfqamount, on = ['code', 'date'], how = 'left')
traindf = mergedf.dropna()
traindf = traindf.sort_values(['code', 'date'])
traindf = ff.take_lag(traindf, 'code')
traindf = traindf.sort_values(['code', 'date'])
traindf = traindf[traindf['code'] == traindf['lag_1_code']]
traindf = ff.take_forward(traindf, 'code', fnum = lag)
traindf = traindf.sort_values(['code', 'date'])
traindf = traindf[traindf['code'] == traindf['f_%s_code'%lag]]
traindf = ff.take_forward(traindf, 'target', fnum = lag)
traindf['f_%s_target'%lag][traindf['canbuy'] == 0] = np.nan
traindf = traindf.sort_values(['code', 'date'])
days = 3
for i in range(1, days + 1):
    for j in range(8):
        traindf = ff.take_lag(traindf, 'buy_vol_%s'%j, i)
        traindf = ff.take_lag(traindf, 'sell_vol_%s'%j, i)
    traindf = ff.take_lag(traindf, 'sumbuy', i)
    traindf = ff.take_lag(traindf, 'sumsell', i)

for i in range(1, days + 1):
    traindf = ff.take_lag(traindf, 'code', lagnum = i)
for i in range(1, days + 1):
    traindf = traindf[traindf['code'] == traindf['lag_%s_code'%i]]

features = []
features.append('pchange')
for i in range(8):
    traindf['buy_rate_%s'%i] = traindf['buy_vol_%s'%i] / traindf['sumsell']
    features.append('buy_rate_%s'%i)
for i in range(8):
    traindf['sell_rate_%s'%i] = traindf['sell_vol_%s'%i] / traindf['sumbuy']
    features.append('sell_rate_%s'%i)
traindf['total_rate'] = traindf['sumbuy'] / traindf['sumsell']
features.append('total_rate')

for j in range(1, days + 1):
    for i in range(8):
        traindf['lag_%s_buy_rate_%s'%(j, i)] = traindf['lag_%s_buy_vol_%s'%(j, i)] / traindf['lag_%s_sumsell'%j]
        features.append('lag_%s_buy_rate_%s'%(j, i))
    for i in range(8):
        traindf['lag_%s_sell_rate_%s'%(j, i)] = traindf['lag_%s_sell_vol_%s'%(j, i)] / traindf['lag_%s_sumbuy'%j]
        features.append('lag_%s_sell_rate_%s'%(j, i))
    traindf['lag_%s_total_rate'%j] = traindf['lag_%s_sumbuy'%j] / traindf['lag_%s_sumsell'%j]
    features.append('lag_%s_total_rate'%j)
    traindf = ff.take_lag(traindf, 'pchange', j)
    features.append('lag_%s_pchange'%j)

dropidx = list(traindf[traindf['sumsell'] == 0].index)
traindf = traindf.drop(dropidx)
dropidx = list(traindf[traindf['sumbuy'] == 0].index)
traindf = traindf.drop(dropidx)

dropidx = list(traindf[traindf['total_rate'] > 5].index)
traindf = traindf.drop(dropidx)

traindf[features] = traindf[features].applymap(ff.replace_inf)
traindf = traindf.dropna()
features.extend(['amplitude', 'above_mean'])

/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Variables Descriptions is as follow

In [9]:
traindf[['buy_rate_0', 'sell_rate_0', 'lag_1_buy_rate_0', 'lag_1_sell_rate_0', 'total_rate', 'pchange']].head()

,buy_rate_0,sell_rate_0,lag_1_buy_rate_0,lag_1_sell_rate_0,total_rate,pchange
1039537,0.002599,0.002960,0.002599,0.002960,1.057931,0.018119
296974,0.004069,0.007870,0.002599,0.002960,0.774878,-0.006867
1039538,0.004069,0.007870,0.004069,0.007870,0.774878,-0.006867
296975,0.009551,0.005967,0.004069,0.007870,1.066063,0.001745
1039539,0.009551,0.005967,0.009551,0.005967,1.066063,0.001745


* The high frequency volume will be divided in a quintile fashion based on the following thresholds: <br />
vollist = [0, 10000, 50000, 100000, 200000, 300000, 400000, 500000, infinity]
* The '0' in 'buy_rate_0' is the ratio of buying volume in [0, 10000] to total selling volume, showing the small traders buying power over total selling power. The relationship is as follow: <br />
'0': [0, 10000] <br />
'1': [10000, 50000] <br />
.<br/>
.<br/>
.<br/>
'6': [400000, 500000] <br />
'7': [500000, infinity] <br />
The larger number will show the larger buying power over total selling power.
* The 'sell_rate_...' is the opposite. It shows the selling power in a certain range over the total buying power.
* 'total_rate' indicate the total buying power over the total selling power
* 'pchange' is the close price of the day / open price of the day - 1
* The 'lag_1' means the feature one day before the trading day. We have 'lag_1', 'lag_2', 'lag_3' in this dataset.

### 3.3 Conclusion on Features
* There are 74 features to train
* The time range of the dataset is from 2017-09-04 to 2018-02-28

In [12]:
cols = features.copy()
cols.append('f_2_target')
traindf2 = traindf[cols]
traindf2.to_csv(data_path + 'traindf.csv')